In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import os
from tqdm import tqdm

In [2]:
data_dir = '../New_Data'
num_classes = 6
batch_size = 32
img_size = 128
epochs = 25
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [4]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = full_dataset.classes

In [5]:
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [6]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, num_classes)
)

In [8]:
model = model.to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

In [10]:
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    print(f"\nEpoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader, desc="Training", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

        loop.set_postfix(loss=loss.item(), acc=100. * train_correct / train_total)

    train_acc = 100. * train_correct / train_total
    print(f"Epoch {epoch+1} Completed | Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")


Epoch 1/25


Epoch 1 Completed | Loss: 183.9305 | Train Acc: 63.23%

Epoch 2/25


Epoch 2 Completed | Loss: 137.0997 | Train Acc: 72.47%

Epoch 3/25


Epoch 3 Completed | Loss: 127.4059 | Train Acc: 74.44%

Epoch 4/25


Epoch 4 Completed | Loss: 122.5427 | Train Acc: 75.43%

Epoch 5/25


...

Epoch 21 Completed | Loss: 95.9564 | Train Acc: 81.13%

Epoch 22/25


Epoch 22 Completed | Loss: 92.0750 | Train Acc: 81.30%

Epoch 23/25


Epoch 23 Completed | Loss: 90.2615 | Train Acc: 81.82%

Epoch 24/25


Epoch 24 Completed | Loss: 92.5773 | Train Acc: 82.27%

Epoch 25/25


Epoch 25 Completed | Loss: 92.3834 | Train Acc: 81.61%


In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

val_acc = correct / total * 100
print(f"\nValidation Accuracy: {val_acc:.2f}%")


Validation Accuracy: 82.44%


In [12]:
torch.save(model.state_dict(), "waste_classifier_resnet18.pth")
print("Model saved as waste_classifier_resnet18.pth")

Model saved as waste_classifier_resnet18.pth


In [13]:
print("Class indices:", dict(zip(class_names, range(num_classes))))

Class indices: {'biodegradable': 0, 'cardboard': 1, 'glass': 2, 'metal': 3, 'paper': 4, 'plastic': 5}
